In [1]:
# Imports
import pandas as pd
import os 
from bs4 import BeautifulSoup as bs
from splinter import Browser
from time import sleep

In [2]:
#Method for scraping the tables from website.
# @param [integer]startingSeason - most recent season you want to get
# @param [integer]numberOfSeasons - number of seasons you want to get
# @param 
def scrapeTables(startingSeason, numberOfSeasons, startingObj):
    dictionary = {}
    for x in range(startingObj, (startingObj - numberOfSeasons), -1):
        element = browser.find_by_name('Season').first
        element.select(f'object:{x}')
        sleep(2)

        #Finding the stats table element in html
        html = browser.html
        soup = bs(html, 'html.parser')
        season_table = soup.table
        season_df = pd.read_html(season_table.prettify(), flavor='bs4')[0]

        #Store each season dataframe in the dictionary
        dictionary[f'{startingSeason}'] = season_df.copy()
        startingSeason += 1
    
    return dictionary

In [3]:
path= os.path.join('ChromeDriver','chromedriver.exe')
executable_path = {'executable_path':path}

browser = Browser('chrome', **executable_path, headless=False)

# Extracting data from the webiste

In [4]:
# Visit the following URL
url = "https://stats.nba.com/"
browser.visit(url)

### Players seasons tables

In [5]:
#Navigate through website the get the stats table of best players from seasons

browser.click_link_by_partial_text('See all Player Stats')
sleep(1)
html = browser.html
soup = bs(html, 'html.parser')

startingSeason = 2000

players_seasons_dict = scrapeTables(startingSeason, 20, 27)

D:\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


### Teams seasons tables 

In [6]:
#Navigate through website the get the stats table of best players from seasons

browser.click_link_by_partial_text('Teams')
sleep(1)

browser.click_link_by_partial_text('See All Stats')
sleep(1)

html = browser.html
soup = bs(html, 'html.parser')

teams_seasons_dict = scrapeTables(startingSeason, 20, 27)

# Modifying the data

In [7]:
for season in range(startingSeason, (startingSeason + 20), 1):
    players_seasons_dict[str(season)] = players_seasons_dict[str(season)][['#', 'Player', 'GP', 'PTS', 'REB', 'AST', 'BLK']]
    teams_seasons_dict[str(season)].rename(columns={'Unnamed: 0':'#'}, inplace=True)
    teams_seasons_dict[str(season)] = teams_seasons_dict[str(season)][['#', 'TEAM', 'GP', 'W', 'L', 'PTS']]

In [8]:
teams_seasons_dict['2000'].head()

,#,TEAM,GP,W,L,PTS
0,1,San Antonio Spurs,82,58,24,96.2
1,2,Los Angeles Lakers,82,56,26,100.6
2,2,Philadelphia 76ers,82,56,26,94.7
3,4,Sacramento Kings,82,55,27,101.7
4,5,Dallas Mavericks,82,53,29,100.5


In [9]:
teams_seasons_dict['2019'].head()

,#,TEAM,GP,W,L,PTS
0,1,Milwaukee Bucks,65,53,12,118.6
1,2,Los Angeles Lakers,63,49,14,114.3
2,3,Toronto Raptors,64,46,18,113.0
3,4,LA Clippers,64,44,20,116.2
4,5,Boston Celtics,64,43,21,113.0
